<a href="https://colab.research.google.com/github/IdrissaMoussaAdamou/Kaggle-Rossman-Store-Sales/blob/main/fork_of_suicide_squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
#idrissa

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# *Analysis*

In [ ]:
# import necessary libraries.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

In [ ]:
# import data. Be careful for Date type, String type (factors in R).

train = pd.read_csv("/kaggle/input/rossmann-store-sales/train.csv", sep=',', parse_dates=['Date'],
                    dtype={'StateHoliday': str, 'SchoolHoliday':str})

test = pd.read_csv("/kaggle/input/rossmann-store-sales/test.csv", sep=",", index_col = 'Id', parse_dates=['Date'],
                  dtype={'StateHoliday': str, 'SchoolHoliday':str})

store = pd.read_csv("/kaggle/input/rossmann-store-sales/store.csv", sep=",", dtype={'StoreType': str,
                                                      'Assortment': str,
                                                      'PromoInterval': str})

**1) Train Dataset**

In [ ]:
train.dtypes

In [ ]:
train.isnull().sum()

In [ ]:
# Inspecting train data - See first rows.
train.head()

In [ ]:
train.tail()

In [ ]:
#train.ix[train['Open'] == 1, 'Sales'].mean()
train[train['Open'] == 0].Sales.describe()

In [ ]:
train['Year'] = pd.DatetimeIndex(train['Date']).year
train['Month'] = pd.DatetimeIndex(train['Date']).month
#train['Day'] = pd.DatetimeIndex(train['Date']).day

In [ ]:
train.head()

In [ ]:
train = train[['Store', 'DayOfWeek', 'Date', 'Year', 'Month', 'Customers', 'Open',
               'Promo', 'StateHoliday', 'SchoolHoliday', 'Sales']]
list(train.columns.values)
#print(train.head())

In [ ]:
train.dtypes

In [ ]:
train["StateHoliday"].unique()

In [ ]:
train["SchoolHoliday"].unique()

In [ ]:
# It worked, now automatize the process.

def factor_to_integer(df, colname, start_value=0):
    while df[colname].dtype == object:
        myval = start_value # factor starts at "start_value".
        for sval in df[colname].unique():
            df.loc[df[colname] == sval, colname] = myval
            myval += 1
        df[colname] = df[colname].astype(int, copy=False)
    print('levels :', df[colname].unique(), '; data type :', df[colname].dtype)

In [ ]:
factor_to_integer(train, 'StateHoliday')


In [ ]:
factor_to_integer(train, 'SchoolHoliday')

In [ ]:
train.dtypes

In [ ]:
print(train.describe())

In [ ]:
train['Open'].unique()

In [ ]:
train = train[['Store', 'DayOfWeek', 'Date', 'Year', 'Month', 'Customers', 'Open',
               'Promo', 'StateHoliday', 'SchoolHoliday', 'Sales']]
list(train.columns.values)

In [ ]:
print("NANs for individual columns")
print("---------------------------")
from collections import Counter
x = {colname : train[colname].isnull().sum() for colname in train.columns}
Counter(x).most_common()

In [ ]:
train.dtypes

In [ ]:
train.isnull().sum()

In [ ]:
corMat = pd.DataFrame(train.loc[:, ['DayOfWeek', 'Sales', 'Month', 'Year', 'Customers', 'Promo',
                                    'StateHoliday', 'SchoolHoliday']].corr())
print(corMat)

In [ ]:
sns.heatmap(data=corMat)
plt.show()

**2) Test Dataset**

In [ ]:
test.dtypes

In [ ]:
test.isnull().sum()

In [ ]:
test['Year'] = pd.DatetimeIndex(test['Date']).year
test['Month'] = pd.DatetimeIndex(test['Date']).month
#test['Day'] = pd.DatetimeIndex(test['Date']).day

In [ ]:
test.head()

In [ ]:
test.dtypes

In [ ]:
test['Open'].unique()

In [ ]:
sum(test['Open'] == 0)

In [ ]:
sum(test['Open'] == 1)

In [ ]:
sum(test['Open'].isnull())

In [ ]:
test = test[['Store', 'DayOfWeek', 'Date', 'Year', 'Month', 'Open',
             'Promo', 'StateHoliday', 'SchoolHoliday']]
list(test.columns.values)

In [ ]:
test.dtypes

In [ ]:
print("NANs for individual columns")
print("---------------------------")
from collections import Counter
x = {colname : test[colname].isnull().sum() for colname in test.columns}
Counter(x).most_common()

In [ ]:
print(test[np.isnan(test['Open'])])

In [ ]:
print(train.loc[train['Store'] == 622].head())

In [ ]:
train[np.isnan(train['Open'])]

In [ ]:
test.loc[np.isnan(test['Open']), 'Open'] = 1

In [ ]:
print("NANs for individual columns")
print("---------------------------")
from collections import Counter
x = {colname : test[colname].isnull().sum() for colname in test.columns}
Counter(x).most_common()

In [ ]:
test.dtypes

In [ ]:
test['StateHoliday'].unique()

In [ ]:
test['SchoolHoliday'].unique()

In [ ]:
factor_to_integer(test, 'StateHoliday')
factor_to_integer(test, 'SchoolHoliday')
test.dtypes

In [ ]:
print(train['StateHoliday'].unique())
print(test['StateHoliday'].unique())

In [ ]:
train.loc[train['StateHoliday'] > 1].shape

In [ ]:
##Because only StateHoliday 0 and 1 exist in test dataset, we should consider deleting the rows in train dataset that 
##the StateHoliday value is different than 0 or 1.

In [ ]:
train = train.loc[train['StateHoliday'] < 2]

In [ ]:
print(train['StateHoliday'].unique())
print(test['StateHoliday'].unique())

In [ ]:
train.loc[train['StateHoliday'] > 1].shape

In [ ]:
train.describe()

In [ ]:
test.dtypes

In [ ]:
test.isnull().sum()

**3) Store Dataset**

In [ ]:
store.dtypes

In [ ]:
store.isnull().sum()

In [ ]:
store.shape

In [ ]:
print(store.head())

In [ ]:
print(store.tail())

In [ ]:
print("NANs for individual columns")
print("---------------------------")
from collections import Counter
x = {colname : store[colname].isnull().sum() for colname in store.columns}
Counter(x).most_common()

In [ ]:
store['PromoInterval'].unique()

In [ ]:
store['Promo2SinceWeek'].unique()

In [ ]:
store['Promo2SinceYear'].unique()

*on doit affecter 0 à (Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval) si il y a pas de promo*

In [ ]:
store.loc[store['Promo2'] == 0, ['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']] = 0

In [ ]:
store.loc[store['Promo2'] == 0, ['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']].tail()

*Comme on veut les durée et non les valeur(Year,Week) on fait le max moins la valeur actuel*

In [ ]:
store.loc[store['Promo2'] != 0, 'Promo2SinceWeek'] = store['Promo2SinceWeek'].max() - store.loc[store['Promo2'] != 0, 'Promo2SinceWeek']

In [ ]:
store.loc[store['Promo2'] != 0, 'Promo2SinceYear'] = store['Promo2SinceYear'].max() - store.loc[store['Promo2'] != 0, 'Promo2SinceYear']


In [ ]:
store.loc[store['Promo2'] == 1, ['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']].tail()

In [ ]:
factor_to_integer(store, 'PromoInterval', start_value=0)

In [ ]:
store.dtypes

In [ ]:
store['StoreType'].unique()

In [ ]:
store['Assortment'].unique()

In [ ]:
factor_to_integer(store, 'StoreType')
factor_to_integer(store, 'Assortment')
store.dtypes

In [ ]:
print(store.head())

In [ ]:
store.isnull().sum()

*on remplace pas les naan par des 0 mais par la moyenne car il se peut qu'ils y ait des déjà 0 dans la Col et il se peut qu'il y ait des lignes avec des 0 et ils se confondront*

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(store)
store_imputed = imputer.transform(store)

In [ ]:
store2 = pd.DataFrame(store_imputed, columns=store.columns.values)

In [ ]:
store2.tail()

In [ ]:
store2.head()

In [ ]:
store2.isnull().sum()

In [ ]:
store2.shape

In [ ]:
store.shape

*même chose que pour les promo*

In [ ]:
store2['CompetitionOpenSinceMonth'] = store2['CompetitionOpenSinceMonth'].max() - store2['CompetitionOpenSinceMonth']

In [ ]:
store2['CompetitionOpenSinceYear'] = store2['CompetitionOpenSinceYear'].max() - store2['CompetitionOpenSinceYear']

In [ ]:
train.head()

In [ ]:
store2.head()

In [ ]:
store2.dtypes

In [ ]:
store2.isnull().sum()

In [ ]:
len(store2['Store']) -sum(store2['Store'].isin(train['Store']))

*Checking if there are additional (unnecessary) stores in "train" data.
No difference at all!*


In [ ]:
StoreStore = pd.Series(store2['Store']); StoreTrain = pd.Series(train['Store'])

sum(StoreTrain.isin(StoreStore) == False)

*Merge train and store datasets before modeling the data*
*le merge se fait en fonction du store*

In [ ]:
train_store = pd.merge(train, store2, how = 'left', on='Store')

In [ ]:
print(train_store.head())

In [ ]:
train_store.isnull().sum()

*Merge test and store datasets and check the result*

In [ ]:
test_store = test.reset_index().merge(store2, how = 'left', on='Store').set_index('Id')

In [ ]:
print(test_store.head())

In [ ]:
print(test_store.tail())

In [ ]:
#test_store dimension
test_store.shape

In [ ]:
test_store.isnull().sum()

# *Visual Exploration*

**Sales by Year**

In [ ]:
train_store.boxplot(column='Sales', by='Year')
plt.show()

**Sales by Month**

In [ ]:
train_store.boxplot(column='Sales', by='Month')
plt.show()

In [ ]:
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,4))
axis1.title.set_text('Year vs.Sales')
axis2.title.set_text('Month vs.Sales')
sns.barplot(x='Year', y='Sales', data=train_store, ax=axis1)
sns.barplot(x='Month', y='Sales', data=train_store, ax=axis2)

**Sales by StateHoliday and SchoolHoliday**

In [ ]:
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,4))
axis1.title.set_text('StateHoliday vs.Sales')
axis2.title.set_text('SchoolHoliday vs.Sales')
sns.barplot(x='StateHoliday', y='Sales', data=train_store, ax=axis1)
sns.barplot(x='SchoolHoliday', y='Sales', data=train_store, ax=axis2)

In [ ]:
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,4))
axis1.title.set_text('StoreType vs.Sales')
axis2.title.set_text('DayofWeek vs.Sales')
sns.barplot(x='StoreType', y='Sales', data=train_store, ax=axis1)
sns.barplot(x='DayOfWeek', y='Sales', data=train_store, ax=axis2)

In [ ]:
fig, (axis1,axis2) = plt.subplots(1,2,figsize=(15,4))
axis1.title.set_text('Promo2 vs.Sales')
axis2.title.set_text('Promo2SinceYear vs.Sales')
sns.barplot(x='Promo2', y='Sales', data=train_store, ax=axis1)
sns.barplot(x='Promo2SinceYear', y='Sales', data=train_store.loc[train_store['Promo2'] != 0], ax=axis2)

**Sales & years**

In [ ]:
train_store.hist(column='Sales', by='Year', bins=30)
plt.show()

**Sales & months**

In [ ]:
train_store.hist(column='Sales', by='Month', bins=30)
plt.show()

**Effect of distance of the nearest store**

In [ ]:
train_store.hist(column='CompetitionDistance', bins=30)
plt.show()

# ***MODELING***

In [ ]:
train_store.columns.values

In [ ]:
test_store.columns.values

In [ ]:
train_model = train_store.drop(['Customers', 'Date'], axis=1)
train_model['Year'] = train_model['Year'].max() - train_model['Year']
print(train_model.head())

In [ ]:
test_model = test_store.drop(['Date'], axis=1)
test_model['Year'] = test_model['Year'].max() - test_model['Year']
print(test_model.head())

In [ ]:
test_model_open = test_model.loc[test_model['Open'] == 1]
test_model_open = test_model_open.drop('Open', axis=1)

test_model_close = test_model.loc[test_model['Open'] == 0]

test_model_open.head()

In [ ]:
test_model_open.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split
X = train_model.drop('Sales', axis=1)
y = train_model['Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = pd.DataFrame(scaler.transform(X_train),
                                  columns=X_train.columns.values)

X_test_scaled = pd.DataFrame(scaler.transform(X_test),
                                  columns=X_train.columns.values)


In [ ]:
X_train2 = X_train
X_train2 = X_train2.drop('CompetitionOpenSinceYear', axis=1)

X_test2 = X_test

X_test2 = X_test2.drop('CompetitionOpenSinceYear', axis=1)

test_model2 = test_model

test_model2 = test_model2.drop('CompetitionOpenSinceYear', axis=1)

In [ ]:
X_train2.shape

In [ ]:
X_train.shape

***Régression linéaire***

In [ ]:
#entrainement du modèle Régression linéaire
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
 
lmodellineaire = LinearRegression()
lmodellineaire.fit(X_train, y_train)
 
# model evaluation for testing set
y_test_predict = lmodellineaire.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, y_test_predict)))
r2 = r2_score(y_test, y_test_predict)
 
print('La performance du modèle sur la base de test')
print('--------------------------------------')
print('L erreur quadratique moyenne est {}'.format(rmse))
print('le score R2 est {}'.format(r2))

Y_test_RL_predict = lmodellineaire.predict(test_model)

In [ ]:
print(Y_test_RL_predict)

**RandomForest**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_jobs=-1, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
rf.score(X_test, y_test)

In [ ]:
Y_test_RF_predict = rf.predict(test_model)
print(Y_test_RF_predict)

**Decission Tree**

In [ ]:
#Fitting Decision Tree Regressor to the training set  
from sklearn.tree import DecisionTreeRegressor
classifier= DecisionTreeRegressor(random_state=44)  
classifier.fit(X_train2, y_train)  
classifier.score(X_test2,y_test)

In [ ]:
Y_test_DT_predict = classifier.predict(test_model2)  
print(Y_test_DT_predict)

In [ ]:
test_model['Sales'] = Y_test_RL_predict
submission1 = test_model['Sales']
submission1 = submission1.reset_index()

#test_store = test.reset_index().merge(store2, how = 'left', on='Store').set_index('Id')

In [ ]:
test_model = test_model.drop(['Sales'], axis=1)

In [ ]:
test_model['Sales'] = Y_test_RF_predict
submission2 = test_model['Sales']
submission2 = submission2.reset_index()

In [ ]:
test_model = test_model.drop(['Sales'], axis=1)

In [ ]:
test_model['Sales'] = Y_test_DT_predict
submission3 = test_model['Sales']
submission3 = submission3.reset_index()

In [ ]:
test_model = test_model.drop(['Sales'], axis=1)

In [ ]:
submission1.tail()

In [ ]:
submission2.tail()

In [ ]:
submission3.tail()

In [ ]:
submission1.to_csv("./submission_open1.csv", sep=',', index=False)

In [ ]:
submission2.to_csv("./submission_open2.csv", sep=',', index=False)

In [ ]:
submission3.to_csv("./submission_open3.csv", sep=',', index=False)

In [ ]:
train_model.to_csv("/kaggle/working/train_model.csv", sep=',', index=False)

In [ ]:
test_model.reset_index().to_csv("./test_model.csv", sep=',', index=False)